## 🤍 DB 키워드
- 로컬 DB에 설치되어 있는 데이터베이스들의 목록보기<br>
show databases;<br><br>
- 워크벤치에서 주석처리<br>
-- selcet * from users;<br><br>
- 데이터베이스 세이프 모드 false 시키기<br>
set sql_safe_updates=0;<br><br>
- alt + enter : 커서줄 실행<br>alt + shift + enter : 선택 블록 실행<br><br>
- 대용량 데이터 처리<br>
SET net_read_timeout = 600;<br>
SET net_write_timeout = 600;

## 🤍SQL(Structured Query Language)
관계형 데이터베이스에서 데이터를 정의, 처리, 제어하는데 사용되는 표준화된 언어입니다. SQL은 주로 데이터베이스 관리 시스템(DBMS)과의 상호 작용을 위해 설계되었습니다.
- 데이터 정의 언어DDL : 데이터베이스의 구조를 정의하고 관리하는 데 사용 (CREATE, ALTER, DROP, TRUNCATE )
- 데이터 처리 언어DML : 데이터를 검색, 삽입, 수정, 삭제하는 데 사용 ( SELECT, INSERT, UPDATE, DELETE )
- 데이터 제어 언어DCL : 데이터베이스에 대한 액세스를 제어하는 데 사용 ( GRANT, REVOKE )
- 트랜잭션 제어 언어TCL : 데이터베이스 내의 트랜잭션을 관리하는데 사용 ( COMMIT, ROLLBACK, SAVEPOINT )

### **1. 데이터 정의 언어 (DDL - Data Definition Language)**
**CREATE:** 데이터베이스 객체를 생성합니다. (**`CREATE TABLE`**, **`CREATE INDEX`** 등)
```sql
create database testdatabase;   --데이터베이스 생성
use testdatabase;               --데이터베이스 사용

CREATE TABLE users (
    id INT AUTO_INCREMENT PRIMARY KEY,      -- PRIMARY KEY 유일성을 보장하고 검색 속도를 향상
                                            -- AUTO_INCREMENT 해당 칼럼의 값이 자동으로 1씩 증가
    username VARCHAR(50) NOT NULL,          -- NOT NULL
    email VARCHAR(100) UNIQUE,              -- UNIQUE
    is_business VARCHAR(10) DEFAULT False,  -- DEFAULT 지정된 기본값을 사용
    age INT CHECK (age >= 18)               -- CHECK 해당 칼럼에 저장될 수 있는 값의 범위나 조건을 지정
    gender ENUM('M','F')                    -- 선택하게함, 더 많은 값 가능

    # FK
    dept_id INT,
    FOREIGN KEY (dept_id) REFERENCES departments(dept_id)
                [ON DELETE CASCADE/ON UPDATE CASCADE]
                 #CASCADE: 참조된 부서가 삭제/변경되면 관련 직원도 같이 삭제/업데이트됨
);
```
**ALTER:** 데이터베이스 객체를 수정합니다. (**`ALTER TABLE`** 등)
```sql
ALTER TABLE employees ADD COLUMN department VARCHAR(50);
```
**DROP:** 데이터베이스 객체를 삭제합니다. (**`DROP TABLE`** 등)
```sql
DROP TABLE employees;
drop database if exists mydatabase;     #mydatabase가 존재하면 삭제
```
**TRUNCATE:** 테이블의 모든 레코드를 삭제하지만 테이블은 유지합니다.
```sql
TRUNCATE TABLE employees;
```

### **2. 데이터 처리 언어 (DML - Data Manipulation Language)**
**SELECT:** 데이터베이스에서 정보를 검색합니다.
> SQL 문법 순서<br>
> SELECT ➡️ FROM ➡️ WHERE ➡️ GROUP BY ➡️ HAVING ➡️ ORDER BY

> SQL 실행 순서<br>
> FROM(JOIN) ➡️ WHERE ➡️ GROUP BY ➡️ HAVING ➡️ SELECT(DISTINCT) ➡️ ORDER BY ➡️ LIMIT/OFFSET
```sql
SELECT * FROM users;        -- *는 SELECT 문에서만 "모든 컬럼"을 의미
SELECT first_name, last_name FROM employees WHERE department = 'IT';
SELECT first_name, last_name FROM B.employees WHERE department = 'IT';      -- 'B'DB의 테이블을 선택

# 추가 컬럼 만들기
SELECT age, age * 100 AS age100 FROM users;     -- AS로 모든 컬럼에 별칭을 줄 수 있다

# 데이터 가공
SELECT DISTINCT age FROM users;                         -- 중복 데이터 삭제
SELECT name, age, CASE WHEN age >= 30 THEN '성인' ELSE '미성년자' END AS age_group FROM users;
    -- CASE(시작) WHEN ... THEN ...(when이 여러개 가능 ','사용X) ELSE ... END는 하나의 완전한 표현식
    -- CASE는 조건에 따른 값을 변환시켜 새로운 컬럼을 만든다

# 순위 부여 ROW_NUMBER()
SELECT name, age, ROW_NUMBER() OVER (ORDER BY age DESC) AS rank FROM users;
SELECT * FROM users ORDER BY age;                       -- 오름차순
SELECT * FROM users ORDER BY age DESC;                  -- 내림차순
SELECT * FROM users ORDER BY age ASC, created DESC;     -- 나이 오름차순 후 생성시간으로 내림차

# JOIN
SELECT users.name, users.age, orders.order_id
FROM users JOIN orders ON users.user_id = orders.user_id;

# WHERE
SELECT * FROM users WHERE age = 30;                     -- ==이 아니라 =
SELECT * FROM users WHERE age = 33 AND name = 'Leo';    -- OR 가능
SELECT * FROM users WHERE NOT age = 33;
SELECT * FROM users WHERE age BETWEEN 20 AND 25;        -- 양쪽 경계값 포함

# GROUP BY
SELECT age, COUNT(*) as user_count FROM users GROUP BY age;

# 페이징 LIMIT
SELECT * FROM users LIMIT 5;            -- 상위 5개
SELECT * FROM users LIMIT 10, 5;        -- 10번째부터 5개의 데이터 조회
```

**INSERT:** 새로운 데이터를 테이블에 삽입합니다.
```sql
INSERT INTO users(username, email, age) VALUES('john_doe', 'john@example.com', 25);
INSERT INTO users SET username = 'john', email = 'john@example.com', age = 25;

# 일부만 값을 지정하면 나머지는 기본값 or NULL
# 여러개의 레코드 한번에 추가 가능
INSERT INTO users(username, email, age)
VALUES
    ('alice', 'alice@example.com', 30),
    ('bob', 'bob@example.com', 28),
    ('charlie', 'charlie@example.com', 35);

# 에러가 있을 때 레코드를 추가하지 않는 IGNORE ( pk, unique, not null, 데이터타입에 오버될 경우 )
INSERT IGNORE INTO users(username, email, age) VALUES('john_doe', 'john@example.com', 25);

# 중복된 값(PK, UNIQUE)이 있는 경우 해당 레코드를 INSERT ➡️ UPDATE
INSERT INTO users(username, email, age) VALUES('john_doe', 'john@example.com', 25)
ON DUPLICATE KEY UPDATE age = age + 2, -- age + 2 같이 현재 값을 기반으로 하는 동적 업데이트도 가능
                        email = VALUES(email); -- 삽입 시도한 email값을 업데이트함
```
** UPDATE: ** 테이블의 기존 데이터를 수정합니다.
```sql
UPDATE 테이블명 SET 컬럼1 = 값1, 컬럼2 = 값2, ... WHERE 조건;
UPDATE users SET name = 'John' WHERE id = 1;
SELECT ROW_COUNT(); -- 업데이트한 후가 아닌 업데이트할 때 같이 실행되어야 바뀐 줄의 개수를 알 수 있다

# CASE를 WHERE절 대신 사용해 여러 조건을 한번에 업데이트
UPDATE products
SET price = CASE
    WHEN stock < 10 THEN price * 1.1
    WHEN stock >= 10 AND stock < 50 THEN price * 1.05
    ELSE price
END;

# LIMIT로 먼저 출력되는 일정 레코드만 업데이트
UPDATE users SET status = 'suspended' ORDER BY RAND() -- ORDER BY RAND()로 무작위로 레코드를 선택
LIMIT 3;

# 서브쿼리
UPDATE products SET price = price * 1.1
WHERE category_id IN(SELECT id FROM categories WHERE name = 'Electronics');
```
** DELETE: ** 테이블에서 데이터를 삭제합니다.
```sql
DELETE FROM employees WHERE employee_id = 1;
DELETE FROM orders WHERE status = 'canceled' LIMIT 100;

#조인하여 삭제
DELETE e FROM employees AS e
JOIN departments AS d ON e.department_id = d.id
WHERE d.name = 'Marketing';

DELETE FROM employees
USING employees, departments    -- using 구문 사용
WHERE employees.department_id = departments.id AND departments.name = 'HR';

DELETE FROM users WHERE age > 65 RETURNING *;   -- 삭제한 행 반환 (PostgreSQL에서 사용 가능)
```

### **3. 데이터 제어 언어 (DCL - Data Control Language)**
- **GRANT:** 사용자에게 특정 작업을 수행할 권한을 부여합니다.
    ```sql
    GRANT SELECT ON employees TO username;
    ```
- **REVOKE:** 사용자로부터 특정 작업 수행 권한을 제거합니다.
    ```sql
    REVOKE SELECT ON employees FROM username;
    ```

#### ︎User(사용자) 생성 및 권한부여
mysql -u root -p # 유저로그인 : 유저와 관련된 작업을 수행하기 위해서는 MySQL에서 관리자(보통 root) 권한이 필요<br>
mysql> USE mysql; # 터미널에서 'mysql'DB 접속<br>

1. mysql 유저 확인<br>
mysql> select * from user;

2. mysql 유저 생성<br>
mysql> CREATE USER 'username'@'localhost' IDENTIFIED BY 'user_password';

3. 사용자 비밀번호 변경<br>
mysql> SET PASSWORD FOR 'username'@'%' = '신규비밀번호';

4. 권한 부여<br>
mysql> GRANT ALL PRIVILEGES ON *.* TO 'username'@'localhost';    #모든 데이터베이스에 대한 권한 부여<br>
mysql> FLUSH PRIVILEGES; # 변경된 권한 적용<br>
mysql> SHOW GRANTS FOR 'username'@'localhost'; # 부여된 권한 확인<br>
mysql> SHOW GRANTS; # 현재 로그인한 유저의 권한 확인

5. 사용자 삭제<br>
mysql> DROP USER 'username'@'%';

> 표현 의미<br>
> 'user'@'localhost' : 로컬에서만 접속 허용<br>
> 'user'@'192.168.0.5' : 특정 IP에서만 접속 허용<br>
> 'user'@'%' : 모든 외부 호스트에서 접속 허용

### 4. **트랜잭션 제어 언어 (TCL-Transaction Control Language)**
- **COMMIT**: 트랜잭션을 완료하고, 데이터베이스 변경사항을 영구적으로 저장
- **ROLLBACK**: 트랜잭션을 취소하고, 마지막 **`COMMIT`** 이후의 모든 변경사항을 되돌림
- **SAVEPOINT**: 트랜잭션 내 특정 지점을 마킹하여 필요시 그 지점으로 되돌릴 수 있음

## 🤍 데이터타입
데이터 타입 공식 문서 : https://dev.mysql.com/doc/refman/8.0/en/data-types.html

숫자형
- BOOLEAN : false 0, true 1 (0이 아닌값은 true로 간주)
- INT(M) 또는 INTEGER(M) : 4바이트 정수
- BIGINT : db에서 AUTO_INCREMENT되면서 매우 많은값이 필요한 경우 int로 부족하기 때문에 bigint를 사용
- DECIMAL(M,D) 또는 NUMERIC(M,D) - 기본값으로 NUMERIC(10, 0)<br>
소수점 포함 최대 M자리 저장(정밀도:전체 자릿수)와 D자리 소수점(스케일)으로 표현. 최대 65자리까지 표현 가능.

문자형
- CHAR(M) : 고정 길이 문자열 저장 (M: 0 ~ 255).
- VARCHAR(M) : 가변 길이 문자열 저장 (M: 0 ~ 65,535).

날짜형
- DATE : 날짜(3바이트), '1000-01-01' ~ '9999-12-31'.
- DATETIME : 날짜와 시간
    - '1000-01-01 00:00:00' ~ '9999-12-31 23:59:59'
    - 8바이트 ( 더 넓은 범위를 커버하며 더 많은 저장 공간을 사용 )
- TIMESTAMP : 날짜와 시간
    - '1970-01-01 00:00:00' ~ '2037-01-19 03:14:07'
    - 4바이트
    - 시간대 인식: TIMESTAMP는 UTC로 저장되며, 조회 시 서버의 시간대 설정에 따라 변환<br>국제화된 앱에서 시간대 변환의 중요성 때문에 유리
    - 자동 갱신: INSERT나 UPDATE 연산 시 현재 시간으로 자동 갱신
```
CREATE TABLE blog_posts (
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP
);
```

<img src="./db2.png" width="800px">